In [1]:
import pandas as pd
import math

In [2]:
# File names and directories
survey_2017_dir = 'J:/Projects/Surveys/HHTravel/Survey2017/Data/Export/Version 2/Public/'
hh_file_name = '2017-pr2-1-household.xlsx'
person_file_name = '2017-pr2-2-person.xlsx'
vehicle_file_name = '2017-pr2-3-vehicle.xlsx'
day_file_name = '2017-pr2-4-day.xlsx'
trip_file_name = '2017-pr2-5-trip.xlsx'
codebook_file_name = '2017-pr2-codebook.xlsx'
codebook_hh_name = '1-HH'
codebook_person_name = '2-PERSON'
codebook_vehicle_name = '3-VEHICLE'
codebook_day_name = '4-DAY'
codebook_trip_name = '5-TRIP'

In [3]:
#How do travel patterns differ by gender?
# by County?
# Mode Choice
# Distance to Work
# Time spent walking and biking
# Travelling with Children/purpose
# income and gender, age and whatever, employment



In [4]:
#merge_hh_person_trips

In [5]:
#apply_weights

In [6]:
#create_cross_tab_with_weights

In [7]:
#start easy, mode choice
hh = pd.read_excel(survey_2017_dir+hh_file_name, skiprows=1)
person= pd.read_excel(survey_2017_dir+person_file_name, skiprows=1)
trip = pd.read_excel(survey_2017_dir+trip_file_name, skiprows=1)


In [8]:
codebook_hh = pd.read_excel(survey_2017_dir+codebook_file_name, skiprows=2, sheet_name = codebook_hh_name)
codebook_person = pd.read_excel(survey_2017_dir+codebook_file_name, skiprows=2, sheet_name = codebook_person_name)
codebook_vehicle = pd.read_excel(survey_2017_dir+codebook_file_name, skiprows=2, sheet_name = codebook_vehicle_name)
codebook_day = pd.read_excel(survey_2017_dir+codebook_file_name, skiprows=2, sheet_name = codebook_day_name)
codebook_trip = pd.read_excel(survey_2017_dir+codebook_file_name, skiprows=2, sheet_name = codebook_trip_name)

In [9]:
def make_codebook(codebook):
    var_names = pd.DataFrame(columns=['Field', 'Variable', 'Value'])
    count = 1
    var_names_dict = []
    for index, row in codebook_hh.iterrows():
        if count == 1:
            last_row = row
        elif row['Field'] == 'Valid Values':
        # the field name comes in the row befor valid values, get it
            field_name = last_row['Field']
            var_names_dict.append({'Field' : field_name, 'Variable': row['Variable'], 'Value':row['Value']})
        elif not(math.isnan(float(row['Variable']))):
        # this happens when your getting another variable value)
             var_names_dict.append({'Field' : field_name, 'Variable': row['Variable'], 'Value':row['Value']})
        last_row = row
        count = count + 1
    var_names = var_names.append(var_names_dict)
    return var_names
    

In [10]:
hh_var_names = make_codebook(codebook_hh)
person_var_names = make_codebook(codebook_person)
vehicle_var_names = make_codebook(codebook_vehicle)
trip_var_names = make_codebook(codebook_trip)

In [11]:
hh_var_names.head()

,Field,Variable,Value
0,hhgroup,1.0,rMove
1,hhgroup,2.0,rSurvey
2,sample_segment,1.0,PSRC - Regular-Low
3,sample_segment,2.0,PSRC - Regular-Medium
4,sample_segment,3.0,PSRC - Regular-High


In [12]:
# join the codebook to the variables??
def merge_hh_person_trip(hh, person,trip):
    hh_person =pd.merge(hh, person, on= 'hhid', suffixes=['', 'person'], how ='right')
    hh_person_trip = pd.merge(hh_person, trip, on= ['hhid', 'personid'], suffixes=['','trip'], how ='right')
    return hh_person_trip

def merge_hh_person(hh, person):
    hh_person =pd.merge(hh, person, on= 'hhid', suffixes=['', 'person'], how ='right')
    return hh_person

person_detail = merge_hh_person(hh, person)
trip_detail = merge_hh_person_trip(hh, person, trip)

In [13]:
drive_trips = trip_detail.loc[(trip_detail['mode_1']>2)& (trip_detail['mode_1']<23)]

In [14]:

drive_trips['origin_seattle'] = 'not origin Seattle'
drive_trips['destination_seattle'] = 'not destination Seattle'

C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
sov_trips = drive_trips.loc[drive_trips['travelers_total']==1]

In [16]:
hov_trips = drive_trips.loc[drive_trips['travelers_total']>1]

In [17]:
hov_trips.loc[(hov_trips['o_puma10']>=11601) & (hov_trips['o_puma10']<=11605), 'origin_seattle']= 'origin in Seattle'

C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [18]:
# Seattle PUMAs 11603, 11602, 11601, 11604, 11605
# that indicates what percentage of workday single-occupant vehicle travel in and out of Seattle originates within the City vs. from elsewhere in the region.

sov_trips['origin_seattle'] = 'not origin Seattle'
sov_trips['destination_seattle'] = 'not destination Seattle'

C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [19]:
sov_trips.loc[(sov_trips['o_puma10']>=11601) & (sov_trips['o_puma10']<=11605), 'origin_seattle']= 'origin in Seattle'


In [20]:
sov_trips.loc[(sov_trips['d_puma10']>=11601) & (sov_trips['d_puma10']<=11605), 'destination_seattle']= 'destination in Seattle'

In [21]:
sov_trips.groupby(['origin_seattle', 'd_rgcname']).sum()['trip_weight_revised'].to_clipboard()

C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\io\clipboards.py:127: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  obj.to_csv(buf, sep=sep, encoding='utf-8', **kwargs)
